# RNN

In [1]:
import numpy as np

## RNN 계층 구현

In [4]:
class RNN:
    def __init__(self, Wx, Wh, b) :
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.cache = None
        
    def forward(self, x, h_prev) :
        Wx, Wh, b = self.params
        t = np.matmul(h_prev, Wh) + np.matmul(x, Wx) + b
        h_next = np.tahn(t)
        
        self.cache = (x, h_prev, h_next)
        return h_next
    
    def backward(self, dh_next) :
        Wx, Wh, b = self.params # 매개변수 2개 지정
        x, h_prev, h_next = self.cache # 중간 데이터 저장
        
        dt = dh_next * (1-h_next ** 2) # tanh 함수 미분 값 * h_next 미분 값
        db = np.sum(dt, axis=0)
        dWh = np.matmul(h_prev.T, dt)
        dh_prev = np.matmul(dt, Wh.T)
        dWx = np.matul(dt, Wx.T)
        
        self.grads[0][...] = dWx
        self.grads[1][...] = dWh
        self.grads[2][...] = db
        
        return dx, dh_prev
    

## Time RNN 계층 구현

In [9]:
class TimeRNN :
    def __init__(self, Wx, Wh, b, stateful=False) :
        self.params = [Wx, Wh, b]
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        self.layers = None # 다수의 RNN 계층을 리스트로 저장
        
        self.h, self.dh = None, None
        self.stateful = stateful
    
    def set_state(self, h) :
        self.h = h
    
    def reset_state(self) :
        self.h = None
    
    def forward(self, xs) :
        Wx, Wh, b = self.params
        N, T, D = xs.shape # 미니배치 크기 N, T개 분량의 시계열, 입력벡터의 차원 수 D
        D, H = Wx.shape # 매개변수 모양
        
        self.layers = []
        hs = np.empty((N,T,H), dtype = 'f') # 출력값을 담을 그릇 준비
        
        if not self.stateful or self.h is None :
            self.h = np.zeros((N,H), dtype = 'f') # stateful도 false라면, self.h가 없다면 영행렬 호출
        
        for t in range(T):
            layer = RNN(*self.params) # t회 RNN 계층 생성 반복
            self.h = layer.forward(xs[:,t,:], self.h) # 처음 self.h는 영행렬이지만 xs가 입력되고 RNN을 거쳐 출력된 값으로 업데이트 됨
            hs[:,t,:] = self.h # t 인덱스 별로 hs 값 입력
            self.layers.append(layer)
            
        return hs # t개 있음
    
    def backward(self, dhs) :
        Wx, Wh, b = self.params
        N, T, D = xs.shape # 미니배치 크기 N, T개 분량의 시계열, 입력벡터의 차원 수 D
        D, H = Wx.shape    
        
        dxs = np.empty((N,T,D), dtype= 'f')
        dh = 0 
        grads = [0,0,0]
        for t in reversed(range(T)) :
            layer = self.layers[t]
            dx, dh = layer.backward(dhs[:,t,:] + dh) # 합산된 기울기
            dxs[:,t,:] = dx
            
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
                
        for i, grad in enumerate(grads) :
            self.grads[i][...] = grad
        self.dh = dh
        return dxs

## RNNLM 구현

In [1]:
import sys
sys.path.append('..')
import numpy as np
from common.time_layers import *

In [42]:
class SimpleRnnlm : 
    def __init__(self, vocab_size, word2vec_size, hidden_size) :
        V,D,H = vocab_size, word2vec_size, hidden_size
        rn = np.random.randn
        
        # 가중치 초기화
        embed_W = (rn(V,D)/100).astype('f')
        rnn_Wx = (rn(D,H)/np.sqrt(D)).astype('f') # Xavier 초깃값
        rnn_Wh = (rn(H,H)/np.sqrt(H)).astype('f') # Xavier 초깃값
        rnn_b = np.zeros(H).astype('f')
        affine_W =  (rn(H,V)/np.sqrt(H)).astype('f')
        affine_b = np.zeros(V).astype('f')
        
        # 계층생성
        self.layers = [
            TimeEmbedding(embed_W),
            TimeRNN(rnn_Wx, rnn_Wh, rnn_b, stateful = True),
            TimeAffine(affine_W, affine_b)
        ]
        self.loss_layer = TimeSoftmaxWithLoss()
        self.rnn_layer = self.layers[1]
        
        # 모든 가중치와 기울기를 리스트에 모은다
        self.params, self.grads = [], []
        for layer in self.layers :
            self.params += layer.params
            self.grads += layer.grads
    
    def forward(self, xs, ts) :
        for layer in self.layers :
            xs = layer.forward(xs)
        loss = self.loss_layer.forward(xs, ts)
        return loss
    
    def backward(self, dout =1) :
        dout = self.loss_layer.backward(dout)
        for layer in reversed(self.layers) :
            dout = layer.backward(dout)
        return dout
    
    def reset_state(self) :
        self.rnn_layer.reset_state()

## RNNLM 학습

In [15]:
import matplotlib.pyplot as plt
from common.optimizer import SGD

In [16]:
import ptb

In [45]:
# 하이퍼파라미터 설정
batch_size = 10
wordvec_size = 100
hidden_size = 100 # RNN 은닉상태 벡터의 원소 수
time_size = 5
lr = 0.1
max_epoch = 100

# 학습데이터 읽기(전체 중 1000개만)
corpus, word_to_id, id_to_word, = ptb.load_data('train')
corpus_size = 1000
corpus = corpus[:corpus_size] # corpus는 1000개, vocab_size는 417+1개
vocab_size = int(max(corpus)+1)

xs = corpus[:-1] # 입력
ts = corpus[1:] # 출력(정답 레이블)
data_size = len(xs)
print('말뭉치 크기: %d, 어휘수 : %d' % (corpus_size, vocab_size))

# 학습시 사용하는 변수
max_iters = data_size//(batch_size*time_size) # 999 // 10 * 5 = 19(몫)
time_idx = 0
total_loss = 0
loss_count = 0
ppl_list = []

# 모델 생성
model = SimpleRnnlm(vocab_size, wordvec_size, hidden_size)
optimizer = SGD(lr)

# 각 미니배치에서 샘플을 읽기 시작 위치를 계산
jump = (corpus_size -1) // batch_size # 999 // 10 = 99
offsets = [i*jump for i in range(batch_size)] # 0, 99, 198 ...

for epoch in range(max_epoch) :
    for iter in range(max_iters) :
        # 미니배치 획득
        batch_x = np.empty((batch_size, time_size), dtype = 'i')
        batch_t = np.empty((batch_size, time_size), dtype = 'i')
        for t in range(time_size) :
            for i, offset in enumerate(offsets) : # (0,0), (1,99) 롤링
                batch_x[i,t] = xs[(offset + time_idx) % data_size] # 나머지값
                batch_t[i,t] = ts[(offset + time_idx) % data_size]
            time_idx +=1
        
        # 기울기를 구하여 매개변수 갱신
        loss = model.forward(batch_x,batch_t)
        model.backward()
        optimizer.update(model.params, model.grads)
        total_loss += loss
        loss_count += 1
        
    # 에폭마다 퍼플렉서티 평가
    ppl = np.exp(total_loss / loss_count)
    print('| 에폭 %d | 퍼플렉서티 %.2f' % (epoch+1, ppl))
    ppl_list.append(float(ppl))
    total_loss, loss_count = 0, 0

말뭉치 크기: 1000, 어휘수 : 418
| 에폭 1 | 퍼플렉서티 400.05
| 에폭 2 | 퍼플렉서티 282.98
| 에폭 3 | 퍼플렉서티 224.57
| 에폭 4 | 퍼플렉서티 215.30
| 에폭 5 | 퍼플렉서티 205.34
| 에폭 6 | 퍼플렉서티 202.05
| 에폭 7 | 퍼플렉서티 199.16
| 에폭 8 | 퍼플렉서티 196.63
| 에폭 9 | 퍼플렉서티 191.75
| 에폭 10 | 퍼플렉서티 192.37
| 에폭 11 | 퍼플렉서티 188.75
| 에폭 12 | 퍼플렉서티 192.40
| 에폭 13 | 퍼플렉서티 190.60
| 에폭 14 | 퍼플렉서티 190.59
| 에폭 15 | 퍼플렉서티 189.50
| 에폭 16 | 퍼플렉서티 185.61
| 에폭 17 | 퍼플렉서티 183.77
| 에폭 18 | 퍼플렉서티 180.23
| 에폭 19 | 퍼플렉서티 180.95
| 에폭 20 | 퍼플렉서티 182.18
| 에폭 21 | 퍼플렉서티 180.06
| 에폭 22 | 퍼플렉서티 177.10
| 에폭 23 | 퍼플렉서티 173.52
| 에폭 24 | 퍼플렉서티 173.89
| 에폭 25 | 퍼플렉서티 172.30
| 에폭 26 | 퍼플렉서티 171.97
| 에폭 27 | 퍼플렉서티 165.42
| 에폭 28 | 퍼플렉서티 165.93
| 에폭 29 | 퍼플렉서티 164.41
| 에폭 30 | 퍼플렉서티 158.37
| 에폭 31 | 퍼플렉서티 157.93
| 에폭 32 | 퍼플렉서티 153.00
| 에폭 33 | 퍼플렉서티 153.38
| 에폭 34 | 퍼플렉서티 148.12
| 에폭 35 | 퍼플렉서티 147.80
| 에폭 36 | 퍼플렉서티 140.60
| 에폭 37 | 퍼플렉서티 136.21
| 에폭 38 | 퍼플렉서티 131.61
| 에폭 39 | 퍼플렉서티 128.67
| 에폭 40 | 퍼플렉서티 123.73
| 에폭 41 | 퍼플렉서티 121.36
| 에폭 42 | 퍼플렉서티 119.87
| 에폭 43 | 퍼플렉서티 109

In [37]:
999 % data_size

0

In [30]:
len(xs)

999

In [29]:
max_iters = data_size//(batch_size*time_size)
max_iters

19

In [58]:
jump = (corpus_size -1) // batch_size
offsets = [i*jump for i in range(batch_size)]
offsets

[0, 99, 198, 297, 396, 495, 594, 693, 792, 891]

In [62]:
for i, offset in enumerate(offsets) :
    print(i, offset)

0 0
1 99
2 198
3 297
4 396
5 495
6 594
7 693
8 792
9 891


In [64]:
xs[100]

76

In [1]:
batch_x

NameError: name 'batch_x' is not defined